In [1]:
#extract sentences and their corresponding trigger tag list from pickle files
#import bengali_word_list
import pickle
file=open('../work_space/august_last/bengali_pickle_latest/bengali_training_and_vocab_for_trigger_22_8.pkl','rb')
train_x=pickle.load(file)
train_y=pickle.load(file)
test_x=pickle.load(file)
test_y=pickle.load(file)
vocab=pickle.load(file)
file.close()

In [2]:
print(train_y[0])
print(train_x[1])

None
[36713, 36713, 36713, 0, 1, 2, 3, 4, 5]


In [3]:
c=0
for tag in test_y:
    if str(tag)!='None':
        c+=1
print(c)

1687


In [4]:
print(len(vocab))

36714


In [5]:
glove_vocab=[]
embedding_vector_list = []
file = open('../work_space/august_last/bengali_pickle_latest/bengali_word_vector_22_8','r')#extract word vectors from fastext
for line in file.readlines():
    row = line.strip().split(' ')
    vocab_word = row[0]
    glove_vocab.append(vocab_word)
    embed_vector = [float(i) for i in row[1:]] # convert to list of float
    embedding_vector_list.append(embed_vector)
file.close()

In [6]:
len(embedding_vector_list)

36714

In [7]:
import numpy as np
embedding_matrix = np.asarray(embedding_vector_list)

In [8]:
embedding_matrix.shape

(36714, 300)

In [9]:
train_data_x=[]
train_data_y=[]
test_data_x=[]
test_data_y=[]
train_data_x= train_x
train_data_y= train_y
test_data_x= test_x
test_data_y= test_y

In [10]:
train_data_x[1000]

[36713, 36713, 71, 376, 645, 645, 660, 113, 661]

In [11]:
len(test_y)

48324

In [12]:
y_total=train_y+test_y

In [13]:
print(len(y_total))

303212


In [18]:
#checking unique event types

uniq_event_type=[]
for k in range(len(y_total)):
    if y_total[k] not in uniq_event_type:
        uniq_event_type.append(y_total[k])

In [19]:
print(len(uniq_event_type))
print(uniq_event_type)

32
[None, u'NORMAL_BOMBING', u'SUICIDE_ATTACK', u'CYCLONE', u'VEHICULAR_COLLISION', u'EARTHQUAKE', u'SHOOT_OUT', u'TERRORIST_ATTACK', u'SURGICAL_STRIKES', u'RIOTS', u'FIRE', u'ARMED_CONFLICTS', u'TSUNAMI', u'TRANSPORT_HAZARDS', u'EPIDEMIC', u'COLD_WAVE', u'PANDEMIC', u'LAND_SLIDE', u'STORM', u'FLOODS', u'DROUGHT', u'AVIATION_HAZARD', u'FOREST_FIRE', u'INDUSTRIAL_ACCIDENT', u'SEISMIC_RISK', u'HEAT_WAVE', u'TRAIN_COLLISION', u'VOLCANO', u'BLIZZARD', u'AVALANCHES', u'HURRICANE', u'TORNADO']


In [16]:
#uniq_event_type[0]=0#to avoide None

In [20]:
#train event types are transformed to numeric value

k=0
train_data_y_numeric=[]
for k in range(len(train_data_y)):
    train_data_y_numeric.append(uniq_event_type.index(train_data_y[k]))

In [19]:
#test event types are transformed to numeric value
'''
k=0
test_data_y_numeric=[]
for k in range(len(test_data_y)):
    test_data_y_numeric.append(uniq_event_type.index(test_data_y[k]))
'''

NameError: name 'test_data_y' is not defined

In [21]:
import numpy as np
y_hot_train=[]
for y_ in train_data_y_numeric:
    y_hot_train.append(np.eye(len(uniq_event_type))[y_])

In [22]:
file=open('../work_space/august_last/bengali_pickle_latest/word_list.pkl','rb')
word_list_train=pickle.load(file)
word_list_test=pickle.load(file)
file.close()

In [23]:
print(len(word_list_train))
print(len(word_list_test))

254888
48324


In [24]:
x_train=train_data_x
y_train=y_hot_train
#x_test=test_data_x
#y_test=y_hot_test
#y_test_actual=test_data_y_numeric
#x_dev=train_data_x[210000:]
#y_dev=y_hot_train[210000:]


In [25]:
import tensorflow as tf
sess = tf.InteractiveSession()

input_length=9
embed_length=300
no_class=len(uniq_event_type)
no_hidden=200
learning_rate=0.001
tf.reset_default_graph()
x=tf.placeholder(tf.int32,[None,input_length])
print(x)
y=tf.placeholder(tf.int32,[None,no_class])
print(y)

w=tf.Variable(tf.random_normal([2*no_hidden,no_class],stddev=0.35,name='weight_out'),trainable=True)
print(w)
#w_embedding=tf.Variable(tf.random_normal([len(vocab),embed_length],stddev=0.35,name='weight_embed'),trainable=True)
#print(w_embedding)
#number_of_layers=3

with tf.name_scope("embedding"):
    w_embedding = tf.Variable(tf.constant(0.0, shape=[len(vocab), embed_length]), trainable=True, name="w_embedding")
    embedding_placeholder = tf.placeholder(tf.float32, [len(vocab), embed_length])
    embedding_init = w_embedding.assign(embedding_placeholder)
    embedded_chars = tf.nn.embedding_lookup(w_embedding,x)

#embedded_chars = tf.nn.embedding_lookup(w_embedding,x)

x_rnn=tf.transpose(embedded_chars,[1,0,2])
x_rnn_shape=tf.reshape(x_rnn,[-1,embed_length])
x_shape_rnn=tf.split(x_rnn_shape,input_length,0)
print(x_shape_rnn)

x_shape_conv=tf.reshape(embedded_chars,[-1,input_length,embed_length,1])
print(x_shape_conv)

filter_size1=2
filter_size2=3
filter_size3=4
no_of_filter=200

def create_wt(shape):
    return tf.Variable(tf.truncated_normal(shape, stddev=0.05))

def create_bt(size):
    return tf.Variable(tf.constant(0.05, shape=[size]))

def create_conv(ip, no_of_filter, conv_filter_size, no_of_channel):
    wt = create_wt(shape=[conv_filter_size,embed_length,1,no_of_filter])
    b = create_bt(size = no_of_filter)
    layer = tf.nn.conv2d(input = ip, filter= wt, strides =[1,1,1,1], padding='VALID')
    layer_conv=tf.nn.relu(layer+b)
    layer=tf.nn.max_pool(value= layer_conv, ksize= [1,input_length-conv_filter_size+1,1,1],strides=[1,1,1,1],padding='VALID' )
    layer = tf.layers.dropout(layer, rate=0.5, training=True)
    print(layer)
    return layer

def create_flatten_layer(layer):
    layer=tf.reshape(layer,[-1,no_of_filter*3])
    return layer

def create_fc_layer(ip, no_input, no_output):
    w_fc= create_wt(shape=[no_input,no_output,])
    b_fc= create_bt(no_output)
    layer=tf.matmul(ip,w_fc)+b_fc
    return(layer)


def multirnn(x):
    
    #embedded_chars = tf.nn.embedding_lookup(w_embedding,x)
    #print(embedded_chars)
    #x_unstack = tf.unstack(embedded_chars,input_length, 1)
    #print(x_unstack)
    #x = tf.unstack(x, window_size, 1)
    #cell = tf.contrib.rnn.BasicLSTMCell(no_hidden)
    #x=tf.transpose(embedded_chars,[1,0,2])
    #x=tf.reshape(x,[-1,embed_length])
    #x=tf.split(x,input_length,0)
    #print(x)
    lstm_f_cell=tf.contrib.rnn.BasicLSTMCell(no_hidden,forget_bias=1.0)
    lstm_f_cell_d=tf.nn.rnn_cell.DropoutWrapper(lstm_f_cell, output_keep_prob=0.5)
    lstm_b_cell=tf.contrib.rnn.BasicLSTMCell(no_hidden,forget_bias=1.0)
    lstm_b_cell_d=tf.nn.rnn_cell.DropoutWrapper(lstm_b_cell, output_keep_prob=0.5)
    outputs, _, _=tf.contrib.rnn.static_bidirectional_rnn(lstm_f_cell_d,lstm_b_cell_d,x,dtype=tf.float32)
    #print('output')
    #print(outputs[4])
    #outputs, _, _=tf.contrib.rnn.static_bidirectional_rnn(fw_cell,bw_cell,x_unstack,dtype=tf.float32)
    #cell=tf.contrib.rnn.MultiRNNCell([cell] * number_of_layers)
    #cell = tf.nn.rnn_cell.DropoutWrapper(cell, output_keep_prob=dropout)
    #outputs, _ =  tf.contrib.rnn.static_rnn(cell, x, dtype=tf.float32)
    #outputs, _, _=tf.contrib.rnn.stack_bidirectional_rnn(fw_cell, bw_cell, x, dtype=tf.float32)
    #print('output size')
    #print(outputs[4])
    logit= [tf.matmul(output,w) for output in outputs]
    #print(logit[4])
    return(outputs[4])

output_filter1= create_conv(ip= x_shape_conv, no_of_filter= 200, conv_filter_size= filter_size1, no_of_channel=1)
#print(output_filter1)
output_filter2= create_conv(ip= x_shape_conv, no_of_filter= 200, conv_filter_size= filter_size2, no_of_channel=1)
#print(output_filter2)
output_filter3= create_conv(ip= x_shape_conv, no_of_filter= 200, conv_filter_size= filter_size3, no_of_channel=1)
#print(output_filter3)
final_vector= tf.concat([output_filter1,output_filter2,output_filter3],0)
print(final_vector)
output_conv_layer=tf.reshape(final_vector,[-1,no_of_filter*3])
print(output_conv_layer)

output_lstm_layer=multirnn(x_shape_rnn)
print(output_lstm_layer)
output_concat= tf.concat([output_conv_layer, output_lstm_layer],1)
print(output_concat)

layer_fc=create_fc_layer(output_concat, no_input = 1000, no_output =len(uniq_event_type))
print(layer_fc)
cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y,logits=layer_fc))#cost calculation
train=tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
y_pred= tf.nn.softmax(layer_fc, name='y_pred')
print(y_pred)
correct_prediction=tf.equal(tf.argmax(y_pred,1),tf.argmax(y,1))
accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

predicted_class=tf.argmax(y_pred,1)
print(predicted_class)
saver = tf.train.Saver()

init = tf.global_variables_initializer()

/home/rs/16CS91R02/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Tensor("Placeholder:0", shape=(?, 9), dtype=int32)
Tensor("Placeholder_1:0", shape=(?, 32), dtype=int32)
<tf.Variable 'Variable:0' shape=(400, 32) dtype=float32_ref>
[<tf.Tensor 'split:0' shape=(?, 300) dtype=float32>, <tf.Tensor 'split:1' shape=(?, 300) dtype=float32>, <tf.Tensor 'split:2' shape=(?, 300) dtype=float32>, <tf.Tensor 'split:3' shape=(?, 300) dtype=float32>, <tf.Tensor 'split:4' shape=(?, 300) dtype=float32>, <tf.Tensor 'split:5' shape=(?, 300) dtype=float32>, <tf.Tensor 'split:6' shape=(?, 300) dtype=float32>, <tf.Tensor 'split:7' shape=(?, 300) dtype=float32>, <tf.Tensor 'split:8' shape=(?, 300) dtype=float32>]
Tensor("Reshape_1:0", shape=(?, 9, 300, 1), dtype=float32)
Tensor("dropout/dropout/mul:0", shape=(?, 1, 1, 200), dtype=float32)
Tensor("dropout_1/dropout/mul:0", shape=(?, 1, 1, 200), dtype=float32)
Tensor("dropout_2/dropout/mul:0", shape=(?, 1, 1, 200), dtype=float32)
Tensor("concat:0", shape=(?, 1, 1, 200), dtype=float32)
Tensor("Reshape_2:0", shape=(?, 600), d

In [26]:
MODEL_PATH = '../work_space/august_last/bengali_pickle_latest/bengali_trigger_model_30_1.ckpt'
saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(init)
    sess.run(embedding_init, feed_dict={embedding_placeholder: embedding_matrix})
 
    batch_size=1000
    #no_of_batch= int(len(x_train)/batch_size)
    epoch=20
    for r in range(epoch):
        ptr=1
        bs=0
        while ptr*batch_size < len(x_train):
            ip,op=x_train[bs:bs+batch_size],y_train[bs:bs+batch_size]
            #ip_v, op_v= x_dev[:2*batch_size], y_dev[:2*batch_size]
            bs+=batch_size
            ip=np.reshape(ip,[batch_size,input_length])
            op=np.reshape(op,[batch_size,no_class])
            ptr+=1
            sess.run(train,feed_dict={x:ip,y:op})
        print(sess.run(cost,feed_dict={x:ip,y:op}))
        #print(sess.run(accuracy,feed_dict={x:ip_v,y:op_v}))

    saver.save(sess, MODEL_PATH)

0.14046808
0.07862905
0.042753864
0.025611687
0.007695614
0.007143285
0.0029586945
0.001159771
0.00073160947
0.0007757211
0.0006764385
0.00048049723
0.0015819152
0.00026286894
0.0007309383
0.00027756227
0.00021502652
0.0018461258
0.00015776925
0.00031081814


In [27]:
#MODEL_PATH = '../work_space/august_last/bengali_pickle_latest/bengali_trigger_model_30_1.ckpt'
import tensorflow as tf
import pickle
import numpy as np
import math
import os
import sys
reload(sys)
sys.setdefaultencoding('utf8')

#tf.reset_default_graph()


output_path='../work_space/august_last/bengali_pickle_latest/test_trigger_output_31_8/'
path='../work_space/august_last/bengali_pickle_latest/test_fixed_window_pikle'

saver = tf.train.Saver()
with tf.Session() as sess:
    saver.restore(sess, MODEL_PATH)
    for filename  in os.listdir(path):
        fullPath = os.path.join(path, filename)
        if os.path.isfile(fullPath):
            test_file=open(fullPath,'rb')
            test_x=pickle.load(test_file)
            test_y=pickle.load(test_file)
            test_z=pickle.load(test_file)
            word_list=pickle.load(test_file)
            x_test=test_x
            y_test=test_y
            acc=0
            ptr=1
            bs=0
            #test_window_size=len(x_test)
            #test_x=np.reshape(x_test,[batch_size,test_window_size,embed_length])
            #test_y=np.reshape(y_test,[batch_size,test_window_size,no_class])
            #acc=sess.run(accuracy,feed_dict={x:test_x,y:test_y})
            tagset=[]
            batch_size=1
            n_batch = int(math.ceil(len(x_test) / batch_size))
            print(n_batch)
            while ptr <= n_batch:
                if ptr*batch_size < len(x_test):
                    ip,op=x_test[bs:bs+batch_size],y_test[bs:bs+batch_size]
                else:
                    ip,op=x_test[bs:],y_test[bs:]
                bs+=batch_size
                ip=np.reshape(ip,[batch_size,input_length])
                #op=np.reshape(op,[batch_size,no_class])
                ptr+=1
                pred=sess.run(predicted_class,feed_dict={x:ip})
                for p1 in pred:
                    tagset.append(p1)
                    #print 'p1 = ', p1
                    #for p2 in p1:
                    #print 'p2 = ', np.asarray(softmax(p2))
               
                #acc+=sess.run(accuracy,feed_dict={x:ip,y:op})
            #print (acc/ptr), len(tagset)
            #print(tagset)
            f_out=open(output_path+filename.replace('pkl','txt'),'w+')
            t=0
            while t<len(tagset):
                f_out.write(str(vocab[x_test[t][4]])+'   '+str(str(y_test[t])+' '+str(uniq_event_type[tagset[t]])+'\n'))
                t+=1
            f_out.close()

INFO:tensorflow:Restoring parameters from ../work_space/august_last/bengali_pickle_latest/bengali_trigger_model_30_1.ckpt


In [28]:
uniq_event_list=[]
for tag in uniq_event_type:
    tag1=str(tag)
    if tag1 not in uniq_event_list:
        uniq_event_list.append(tag1)

In [29]:
import os
total_tag_trigger_actual=[]
total_tag_trigger_predicted=[]
path='../work_space/august_last/bengali_pickle_latest/test_trigger_output_31_8/'
for filename  in os.listdir(path):
    fullPath = os.path.join(path, filename)
    if os.path.isfile(fullPath):
        actual_tag=[]
        predicted_tag=[]
        f1=open(fullPath,'r')
        for line in f1:
            a=line.split()
            actual_tag.append(str(a[1]))
            predicted_tag.append(str(a[2]))
        total_tag_trigger_actual=total_tag_trigger_actual + actual_tag
        total_tag_trigger_predicted=total_tag_trigger_predicted + predicted_tag
        f1.close()
        
total_predicted_trigger_list=[]
for k in total_tag_trigger_predicted:
    if k=='0':
        total_predicted_trigger_list.append('None')
    else:
        total_predicted_trigger_list.append(k)

In [30]:
f_result=open('../work_space/august_last/bengali_pickle_latest/test_trigger_output_31_8/result_analysis.txt','w+')
for k in uniq_event_list:
    i=0
    tp=0
    total_count=0
    for i in range(len(total_tag_trigger_actual)):
        if total_tag_trigger_actual[i]==k:
            total_count+=1
            if total_predicted_trigger_list[i]==k:
                tp+=1
    fp=0
    i=0
    for i in range(len(total_tag_trigger_actual)):
        if total_predicted_trigger_list[i]==k:
            if total_tag_trigger_actual[i]!=k:
                fp+=1
    fn=0
    i=0
    for i in range(len(total_tag_trigger_predicted)):
        if total_predicted_trigger_list[i]!=k:
            if total_tag_trigger_actual[i]==k:
                fn+=1
    f_result.write(k+' '+'TP='+str(tp)+' '+' FP='+str(fp)+' '+' FN='+str(fn)+'\n')
f_result.close()